## Introduction

In the realm of political campaigns and elections, financial contributions play a pivotal role in shaping outcomes and influencing voter behavior. This analysis focuses on the examination of contributions made to various political committees, utilizing a dataset that encapsulates individual donors, their occupations, and the amounts contributed. By employing SQL queries within a Spark DataFrame framework, we aim to uncover significant patterns and trends in the data, such as the identities of major contributors and the total funding received by different committees. Understanding these dynamics is crucial for grasping how financial resources are allocated in political contexts and their potential impact on electoral processes. This assignment seeks to provide insights into the sources of funding, the distribution of contributions among committees, and the implications of these findings for stakeholders involved in political campaigns.

**Create a new folder, set as working directory and navigate to the folder**

In [1]:
!mkdir Assignment_3

mkdir: cannot create directory ‘Assignment_3’: File exists


In [2]:
%cd /home/ubuntu/Assignment_3

/home/ubuntu/Assignment_3


In [3]:
!pwd

/home/ubuntu/Assignment_3


## Setup the Spark Environment

In [4]:
import findspark

In [5]:
findspark.init()

In [6]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

In [7]:
# Create a SparkContext and SparkSession
sc = SparkContext(appName='Contributions_Analysis')
spark = SparkSession.builder.appName("Contributions Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/26 00:26:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Property used to format output tables better
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


In [9]:
spark

To get started, we identify the data we want to work with, in this case one CSV file and two txt files.

## Data Processing

**Preparing the data**

To be done outside juptyter:
1. Download bulk data from https://www.fec.gov/data/browse-data/?tab=bulk-data
2. Extract files for the period from June to July, 2024
3. Zip the files

Upload the zipped file with June and July data from computer to jupyter in a folder named itcont_20240620_20240723.zip

In [10]:
print("\nAll files in wd before extracting data:")
!ls


All files in wd before extracting data:
202406.csv		 itcont_20240620_20240723.zip
202407.csv		 itcont_2024_20240620_20240709.txt
combined_data.csv	 itcont_2024_20240710_20240723.txt
indiv_header_file.csv	 spark-warehouse
indiv_header_file.csv.1


In [11]:
print("\nExtracting files:")
!unzip -q itcont_20240620_20240723.zip


Extracting files:
replace itcont_2024_20240620_20240709.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

24/11/26 00:26:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


^C


In [12]:
print("\nChecking files were extracted:")
!ls -lh itcont*


Checking files were extracted:
-rw-rw-r-- 1 ubuntu ubuntu 134M Nov 24 13:36 itcont_20240620_20240723.zip
-rw-rw-r-- 1 ubuntu ubuntu 348M Nov 19 11:27 itcont_2024_20240620_20240709.txt
-rw-rw-r-- 1 ubuntu ubuntu 343M Nov 19 11:27 itcont_2024_20240710_20240723.txt


In [13]:
print("\nCounting number of obbservatons in extracted files:")
!wc -l itcont_2024_20240620_20240709.txt itcont_2024_20240710_20240723.txt


Counting number of obbservatons in extracted files:
  1944602 itcont_2024_20240620_20240709.txt
  1902192 itcont_2024_20240710_20240723.txt
  3846794 total


The DataFrame contains 3,846,794 records of individual contributions

In [14]:
print("\nConverting the TXT pipe-delimited data into CSV:")
!csvformat -d "|" itcont_2024_20240620_20240709.txt > 202406.csv
!csvformat -d "|" itcont_2024_20240710_20240723.txt > 202407.csv


Converting the TXT pipe-delimited data into CSV:


**Downloading the header**

In [15]:
print("\nDownloading the header file:")
!wget https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv


--2024-11-25 19:27:57--  https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv
Resolving www.fec.gov (www.fec.gov)... 3.162.125.66, 3.162.125.67, 3.162.125.119, ...
Connecting to www.fec.gov (www.fec.gov)|3.162.125.66|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/data_dictionaries/indiv_header_file.csv [following]
--2024-11-25 19:27:57--  https://cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com/bulk-downloads/data_dictionaries/indiv_header_file.csv
Resolving cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)... 108.175.48.196
Connecting to cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com (cg-519a459a-0ea3-42c2-b7bc-fa1143481f74.s3-us-gov-west-1.amazonaws.com)|108.175.48.196|:443... conne

In [16]:
print("\nChecking the column headers")
!head -n 1 indiv_header_file.csv


Checking the column headers
CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID


**Data Inpsection**

In [17]:
!csvcut -n 202406.csv

  1: C00849273
  2: T
  3: TER
  4: P2024
  5: 202407159660924932
  6: 22Y
  7: IND
  8: REPASS, JAMES
  9: HILLSBORO
 10: VA
 11: 20132
 12: 
 13: 
 14: 06202024
 15: 400
 16: 
 17: SB28A.5248
 18: 1802122
 19: 
 20: 
 21: 4071620241974571456


In [18]:
!csvcut -n 202407.csv

  1: C00217471
  2: N
  3: M8
  4: P
  5: 202408079666110989
  6: 15
  7: IND
  8: HOUGHTON, DENNIS J
  9: VIRGINIA BEACH
 10: VA
 11: 234524607
 12: MAERSK LINE, LTD
 13: DIRECTOR
 14: 07102024
 15: 20
 16: 
 17: SA11AI.14049
 18: 1808452
 19: 
 20: 
 21: 4081520242013125256


In [19]:
print("\nHeader Addition onto combined csv files:")
!cat indiv_header_file.csv 202406.csv 202407.csv > combined_data.csv


Header Addition onto combined csv files:


In [20]:
print("\nVerifying number of lines in combined file ")
!wc -l combined_data.csv


Verifying number of lines in combined file 
3846795 combined_data.csv


In [21]:
print("\nDisplaying the first 5 lines of the combined file to verify it was created successfully:")
!head -n 5 combined_data.csv | csvlook


Displaying the first 5 lines of the combined file to verify it was created successfully:
| CMTE_ID   | AMNDT_IND | RPT_TP | TRANSACTION_PGI |               IMAGE_NUM | TRANSACTION_TP | ENTITY_TP | NAME                   | CITY       | STATE | ZIP_CODE | EMPLOYER       | OCCUPATION | TRANSACTION_DT | TRANSACTION_AMT | OTHER_ID | TRAN_ID         |  FILE_NUM | MEMO_CD | MEMO_TEXT |                    SUB_ID |
| --------- | --------- | ------ | --------------- | ----------------------- | -------------- | --------- | ---------------------- | ---------- | ----- | -------- | -------------- | ---------- | -------------- | --------------- | -------- | --------------- | --------- | ------- | --------- | ------------------------- |
| C00849273 | T         | TER    | P2024           | 202,407,159,660,924,932 | 22Y            | IND       | REPASS, JAMES          | HILLSBORO  | VA    |   20,132 |                |            |      6,202,024 |             400 |          | SB28A.5248      | 1,802,122

In [22]:
print("\nLooking at a sample to detremine domain and range:")
!head -n 1000 combined_data.csv | csvstat


Looking at a sample to detremine domain and range:
  1. "CMTE_ID"

	Type of data:          Text
	Contains null values:  False
	Non-null values:       999
	Unique values:         2
	Longest value:         9 characters
	Most common values:    C00828541 (998x)
	                       C00849273 (1x)

  2. "AMNDT_IND"

	Type of data:          Text
	Contains null values:  False
	Non-null values:       999
	Unique values:         2
	Longest value:         1 characters
	Most common values:    A (998x)
	                       T (1x)

  3. "RPT_TP"

	Type of data:          Text
	Contains null values:  False
	Non-null values:       999
	Unique values:         2
	Longest value:         3 characters
	Most common values:    M7 (998x)
	                       TER (1x)

  4. "TRANSACTION_PGI"

	Type of data:          Text
	Contains null values:  False
	Non-null values:       999
	Unique values:         2
	Longest value:         5 characters
	Most common values:    P2024 (988x)
	                       

In [23]:
!head combined_df.csv | qsv table

head: cannot open 'combined_df.csv' for reading: No such file or directory


## Creating Dataframes for Spark ##

In [25]:
# Load the Header CSV File
header_df = spark.read.option("header", "false").csv("indiv_header_file.csv")
header = header_df.collect()[0]  # The header is in the first row
header_list = [str(col) for col in header]  # Convert each column name to a string

In [26]:
# Load the Combined Data using spark.read.option()
#data = spark.read.option("header", "false").csv('combined_data.csv')
# Load the Combined Data using spark.read.option()
df1 = spark.read.option("header", "false").csv('202406.csv')
df2 = spark.read.option("header", "false").csv('202407.csv')

In [27]:
# Combine DataFrames Using Union
combined_df = df1.union(df2)

In [28]:
# Check the number of columns in combined_df and header_list
print(f"Number of columns in combined_df: {len(combined_df.columns)}")
print(f"Number of columns in header_list: {len(header_list)}")

Number of columns in combined_df: 21
Number of columns in header_list: 21


In [29]:
# Apply the Header to the Combined DataFrame
combined_df = combined_df.toDF(*header_list)

In [30]:
# Show 5 observations
combined_df.show(5)

+---------+---------+------+---------------+------------------+--------------+---------+--------------------+----------+-----+--------+--------------+----------+--------------+---------------+--------+---------------+--------+-------+---------+-------------------+
|  CMTE_ID|AMNDT_IND|RPT_TP|TRANSACTION_PGI|         IMAGE_NUM|TRANSACTION_TP|ENTITY_TP|                NAME|      CITY|STATE|ZIP_CODE|      EMPLOYER|OCCUPATION|TRANSACTION_DT|TRANSACTION_AMT|OTHER_ID|        TRAN_ID|FILE_NUM|MEMO_CD|MEMO_TEXT|             SUB_ID|
+---------+---------+------+---------------+------------------+--------------+---------+--------------------+----------+-----+--------+--------------+----------+--------------+---------------+--------+---------------+--------+-------+---------+-------------------+
|C00849273|        T|   TER|          P2024|202407159660924932|           22Y|      IND|       REPASS, JAMES| HILLSBORO|   VA|   20132|          NULL|      NULL|      06202024|            400|    NULL|    

In [31]:
# Perform Initial Data Exploration
print(f"Total number of records in combined DataFrame: {combined_df.count()}")
combined_df.select('TRANSACTION_AMT').distinct().show()

Total number of records in combined DataFrame: 3846794


+---------------+
|TRANSACTION_AMT|
+---------------+
|             -4|
|            675|
|            691|
|            296|
|           3210|
|           1436|
|          -2024|
|           -757|
|          91421|
|            467|
|            125|
|            800|
|            944|
|          66000|
|          -3200|
|            -30|
|           2700|
|           3200|
|            666|
|           -348|
+---------------+
only showing top 20 rows



## Data Analysis Part 1: RDD Processing

In [32]:
print("loading data into a Spark RDD:")
rdd = combined_df.rdd

loading data into a Spark RDD:


In [35]:
%time rdd.count()

CPU times: user 25.6 ms, sys: 8.74 ms, total: 34.3 ms
Wall time: 1min 18s


3846794

In [36]:
print("Checking header in RDD:")
header = rdd.first()
header

Checking header in RDD:


Row(CMTE_ID='C00849273', AMNDT_IND='T', RPT_TP='TER', TRANSACTION_PGI='P2024', IMAGE_NUM='202407159660924932', TRANSACTION_TP='22Y', ENTITY_TP='IND', NAME='REPASS, JAMES', CITY='HILLSBORO', STATE='VA', ZIP_CODE='20132', EMPLOYER=None, OCCUPATION=None, TRANSACTION_DT='06202024', TRANSACTION_AMT='400', OTHER_ID=None, TRAN_ID='SB28A.5248', FILE_NUM='1802122', MEMO_CD=None, MEMO_TEXT=None, SUB_ID='4071620241974571456')

In [37]:
# Inspect the RDD structure
print("Sample RDD data:")
print(rdd.take(5))

Sample RDD data:
[Row(CMTE_ID='C00849273', AMNDT_IND='T', RPT_TP='TER', TRANSACTION_PGI='P2024', IMAGE_NUM='202407159660924932', TRANSACTION_TP='22Y', ENTITY_TP='IND', NAME='REPASS, JAMES', CITY='HILLSBORO', STATE='VA', ZIP_CODE='20132', EMPLOYER=None, OCCUPATION=None, TRANSACTION_DT='06202024', TRANSACTION_AMT='400', OTHER_ID=None, TRAN_ID='SB28A.5248', FILE_NUM='1802122', MEMO_CD=None, MEMO_TEXT=None, SUB_ID='4071620241974571456'), Row(CMTE_ID='C00828541', AMNDT_IND='A', RPT_TP='M7', TRANSACTION_PGI='P2024', IMAGE_NUM='202408159666207641', TRANSACTION_TP='15', ENTITY_TP='IND', NAME='MANSFIELD, RISA', CITY='COLUMBUS', STATE='MS', ZIP_CODE='39705', EMPLOYER='RETIRED', OCCUPATION='RETIRED', TRANSACTION_DT='06202024', TRANSACTION_AMT='125', OTHER_ID=None, TRAN_ID='SA17A.121694463', FILE_NUM='1810271', MEMO_CD=None, MEMO_TEXT=None, SUB_ID='4081620242013476924'), Row(CMTE_ID='C00828541', AMNDT_IND='A', RPT_TP='M7', TRANSACTION_PGI='P2024', IMAGE_NUM='202408159666207641', TRANSACTION_TP='15

looks like all data was loaded successfully

In [38]:
from operator import add

In [39]:
# Handle potential invalid or missing TRANSACTION_AMT
def safe_float(value):
    try:
        return float(value)
    except (TypeError, ValueError):
        return 0

**The top 10 states in terms of the total count of contributions**

In [40]:
# 1. Top 10 States by Contribution Count
top10_states_count = (
    rdd
    .map(lambda row: (row.STATE, 1))  # Map STATE as key and count each contribution
    .reduceByKey(add)  # Sum counts per state
    .takeOrdered(10, key=lambda pair: -pair[1])  # Take top 10 states by contribution count
)

In [41]:
# Print results
print("\nTop 10 States by Contribution Count:")
for state, count in top10_states_count:
    print(f"{state}: {count}")


Top 10 States by Contribution Count:
CA: 566727
TX: 324357
FL: 279718
NY: 226242
WA: 140883
PA: 130394
VA: 125630
IL: 125190
AZ: 115101
OH: 109589


**Intepretation**

California (CA) has the highest number of contributions, with 566,727 contributions recorded. This indicates a strong engagement or activity in political contributions within the state.

Texas (TX) follows with 324,357 contributions, which is significantly lower than California but still represents a substantial amount of political activity.

Other states like Florida (FL) and New York (NY) also show high levels of contributions, indicating that these states are major players in political financing.

The overall trend suggests that states with larger populations or more active political environments tend to have higher contribution counts.

**The top 10 states in terms of the total (i.e., sum) amount of contributions**

In [42]:
# 2. Top 10 States by Total Amount of Contributions
top10_states_amount = (
    rdd
    .map(lambda row: (row.STATE, safe_float(row.TRANSACTION_AMT)))  # Map STATE as key and use TRANSACTION_AMT as value
    .reduceByKey(add)  # Sum amounts per state
    .takeOrdered(10, key=lambda pair: -pair[1])  # Take top 10 states by total amount
)

In [43]:
# Print results
print("\nTop 10 States by Total Amount of Contributions:")
for state, total_amount in top10_states_amount:
    print(f"{state}: ${total_amount:,.2f}")


Top 10 States by Total Amount of Contributions:
CA: $150,899,253.00
NY: $94,487,440.00
DC: $85,847,491.00
FL: $72,203,347.00
TX: $65,958,686.00
MA: $62,463,680.00
WY: $58,379,042.00
VA: $50,602,434.00
IL: $31,544,698.00
PA: $27,377,532.00


**Interpretation**   
California (CA) again leads with a total contribution amount of $150,899,253, which indicates that not only does it have the highest number of contributions but also the highest financial impact.
    
New York (NY) comes in second with nearly $94 million, suggesting that while it may not have as many individual contributions as California, those contributions tend to be larger on average.
    
The presence of Washington D.C. (DC) in this list highlights its role as a significant hub for political donations despite being a smaller area geographically and demographically.
    
States like Florida (FL) and Texas (TX) also show substantial total amounts contributed but rank lower than their contribution counts might suggest due to potentially smaller average contribution sizes.

## Data Analysis Part 2: Data Frame API

In [77]:
!head combined_data.csv | csvcut -n

  1: CMTE_ID
  2: AMNDT_IND
  3: RPT_TP
  4: TRANSACTION_PGI
  5: IMAGE_NUM
  6: TRANSACTION_TP
  7: ENTITY_TP
  8: NAME
  9: CITY
 10: STATE
 11: ZIP_CODE
 12: EMPLOYER
 13: OCCUPATION
 14: TRANSACTION_DT
 15: TRANSACTION_AMT
 16: OTHER_ID
 17: TRAN_ID
 18: FILE_NUM
 19: MEMO_CD
 20: MEMO_TEXT
 21: SUB_ID


In [78]:
!head -5 combined_data.csv | csvlook

| CMTE_ID   | AMNDT_IND | RPT_TP | TRANSACTION_PGI |               IMAGE_NUM | TRANSACTION_TP | ENTITY_TP | NAME                   | CITY       | STATE | ZIP_CODE | EMPLOYER       | OCCUPATION | TRANSACTION_DT | TRANSACTION_AMT | OTHER_ID | TRAN_ID         |  FILE_NUM | MEMO_CD | MEMO_TEXT |                    SUB_ID |
| --------- | --------- | ------ | --------------- | ----------------------- | -------------- | --------- | ---------------------- | ---------- | ----- | -------- | -------------- | ---------- | -------------- | --------------- | -------- | --------------- | --------- | ------- | --------- | ------------------------- |
| C00849273 | T         | TER    | P2024           | 202,407,159,660,924,932 | 22Y            | IND       | REPASS, JAMES          | HILLSBORO  | VA    |   20,132 |                |            |      6,202,024 |             400 |          | SB28A.5248      | 1,802,122 |         |           | 4,071,620,241,974,571,456 |
| C00828541 | A         | M7     | P2

In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType
from pyspark.sql.functions import to_date, col

# Define the corrected schema to match the website specifications
schema = StructType([
    StructField("CMTE_ID", StringType(), False),        # Non-nullable Filer identification number
    StructField("AMNDT_IND", StringType(), True),       # Amendment indicator
    StructField("RPT_TP", StringType(), True),          # Report type
    StructField("TRANSACTION_PGI", StringType(), True), # Primary-general indicator
    StructField("IMAGE_NUM", StringType(), True),       # Image number
    StructField("TRANSACTION_TP", StringType(), True),  # Transaction type
    StructField("ENTITY_TP", StringType(), True),       # Entity type
    StructField("NAME", StringType(), True),            # Contributor/Lender/Transfer Name
    StructField("CITY", StringType(), True),            # City
    StructField("STATE", StringType(), True),           # State
    StructField("ZIP_CODE", StringType(), True),        # ZIP code
    StructField("EMPLOYER", StringType(), True),        # Employer
    StructField("OCCUPATION", StringType(), True),      # Occupation
    StructField("TRANSACTION_DT", StringType(), True),    # Transaction date (MMDDYYYY)
    StructField("TRANSACTION_AMT", DecimalType(14, 2), True), # Transaction amount
    StructField("OTHER_ID", StringType(), True),        # Other identification number
    StructField("TRAN_ID", StringType(), True),         # Transaction ID
    StructField("FILE_NUM", LongType(), True),          # File number / Report ID
    StructField("MEMO_CD", StringType(), True),         # Memo code
    StructField("MEMO_TEXT", StringType(), True),       # Memo text
    StructField("SUB_ID", LongType(), False)            # Non-nullable FEC record number
])

In [83]:
# Apply schema to the RDD and recreate the DataFrame
contributions = spark.read.csv("combined_data.csv", header=True, schema=schema)

# Convert TRANSACTION_DT to DateType
contributions = contributions.withColumn(
    "TRANSACTION_DT",
    to_date(col("TRANSACTION_DT"), "MMddyyyy")
)

# Convert TRANSACTION_AMT to DecimalType
contributions = contributions.withColumn(
    "TRANSACTION_AMT",
    col("TRANSACTION_AMT").cast(DecimalType(14, 2))
)

# Check the new schema
contributions.printSchema()

root
 |-- CMTE_ID: string (nullable = true)
 |-- AMNDT_IND: string (nullable = true)
 |-- RPT_TP: string (nullable = true)
 |-- TRANSACTION_PGI: string (nullable = true)
 |-- IMAGE_NUM: string (nullable = true)
 |-- TRANSACTION_TP: string (nullable = true)
 |-- ENTITY_TP: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- ZIP_CODE: string (nullable = true)
 |-- EMPLOYER: string (nullable = true)
 |-- OCCUPATION: string (nullable = true)
 |-- TRANSACTION_DT: date (nullable = true)
 |-- TRANSACTION_AMT: decimal(14,2) (nullable = true)
 |-- OTHER_ID: string (nullable = true)
 |-- TRAN_ID: string (nullable = true)
 |-- FILE_NUM: long (nullable = true)
 |-- MEMO_CD: string (nullable = true)
 |-- MEMO_TEXT: string (nullable = true)
 |-- SUB_ID: long (nullable = true)



### Data Analysis

**The contributor’s name, the committee ID the contributor contributed to, and
the transaction amount for all contributions above $5000**

In [84]:
# Contributions above $5000
contributions.filter('TRANSACTION_AMT > 5000') \
    .select('NAME', 'CMTE_ID', 'TRANSACTION_AMT') \
    .orderBy('TRANSACTION_AMT', ascending=False) \
    .show(10)

+--------------------+---------+---------------+
|                NAME|  CMTE_ID|TRANSACTION_AMT|
+--------------------+---------+---------------+
|     MELLON, TIMOTHY|C00825851|    50000000.00|
|          ONE NATION|C00571703|    18400000.00|
|SECURING AMERICAN...|C00881805|    15000000.00|
|FUTURE FORWARD US...|C00669259|    15000000.00|
|CLEMENT, CHRISTIN...|C00857128|    12000000.00|
|BLOOMBERG, MICHAE...|C00495028|    10000000.00|
|     SINGER, PAUL E.|C00504530|    10000000.00|
|SINGER, PAUL ELLIOTT|C00571703|    10000000.00|
|BRICK BY BRICK FO...|C00631549|     5000000.00|
|AMERICAN ACTION N...|C00504530|     5000000.00|
+--------------------+---------+---------------+
only showing top 10 rows



**The name of the individual and their total contributions. Order the records by
the total transaction amount in descending order. Only show the top 10 rows**

**Interpretation**

The highest contribution is from Timothy Mellon, amounting to $50 million.
Significant contributions are also made by organizations like ONE NATION and SECURING AMERICAN DEMOCRACY, indicating strong financial backing for their activities.
The presence of both individual contributors and organizations suggests a mix of funding sources.

In [85]:
# Total contributions per individual
contributions.groupBy("NAME") \
    .agg({"TRANSACTION_AMT": "sum"}) \
    .withColumnRenamed("sum(TRANSACTION_AMT)", "TOTAL_CONTRIBUTIONS") \
    .select("NAME", "TOTAL_CONTRIBUTIONS") \
    .orderBy('TOTAL_CONTRIBUTIONS', ascending=False) \
    .show(10)

+--------------------+-------------------+
|                NAME|TOTAL_CONTRIBUTIONS|
+--------------------+-------------------+
|     MELLON, TIMOTHY|        55003300.00|
|          ONE NATION|        18400000.00|
|FUTURE FORWARD US...|        15190058.00|
|SECURING AMERICAN...|        15000000.00|
|          MUSK, ELON|        14950000.00|
|CLEMENT, CHRISTIN...|        12000564.00|
|     SINGER, PAUL E.|        10150000.00|
|BLOOMBERG, MICHAE...|        10014900.00|
|SINGER, PAUL ELLIOTT|        10000000.00|
|AMERICAN ACTION N...|         7500000.00|
+--------------------+-------------------+
only showing top 10 rows



**Interpretation**

Again, Timothy Mellon leads with total contributions over $55 million, showing a significant commitment to funding.
    
It is worth noting  that Elon Musk also contributed nearly $15 million, indicating his support for certain causes.
The data reflects a diverse range of contributors with varying levels of financial support.

## Data Analysis Part 3: SQL with Data Frame

**Register the contributions DataFrame as a temporary view**

In [86]:
contributions.createOrReplaceTempView("contributions")

**The individual contributor’s name, the contributor's occupation, and the
transaction amount. Order the records by the transaction amount in
descending order.**

In [87]:
spark.sql( """
    SELECT NAME, OCCUPATION, TRANSACTION_AMT
    FROM contributions
    WHERE OCCUPATION IS NOT NULL  -- Ensures we only get contributors with an occupation
    ORDER BY TRANSACTION_AMT DESC
""").show(10)

+--------------------+--------------------+---------------+
|                NAME|          OCCUPATION|TRANSACTION_AMT|
+--------------------+--------------------+---------------+
|     MELLON, TIMOTHY|         INVESTMENTS|    50000000.00|
|CLEMENT, CHRISTIN...|       SELF EMPLOYED|    12000000.00|
|SINGER, PAUL ELLIOTT|           PRESIDENT|    10000000.00|
|BLOOMBERG, MICHAE...|             FOUNDER|    10000000.00|
|     SINGER, PAUL E.|           PRESIDENT|    10000000.00|
|     MELLON, TIMOTHY|         INVESTMENTS|     5000000.00|
|          MUSK, ELON|                 CEO|     5000000.00|
|   BIGELOW, ROBERT T|AEROSPACE & REAL ...|     5000000.00|
|          MUSK, ELON|                 CEO|     5000000.00|
|    MCMAHON, LINDA E|           EXECUTIVE|     5000000.00|
+--------------------+--------------------+---------------+
only showing top 10 rows



**Interpretation**
  
**Top Contributors** :  
Timothy Mellon appears twice with significant contributions of $55 million, indicating he is a major contributor.
Christina Clement and Paul E. Singer are also notable contributors.

**Occupations** :  
The occupations listed provide context about the contributors:
Timothy Mellon is into Investments. Christina Clement is self employed. Paul E. Singer holds the title of "President." Michael Bloomberg is identified as a "Founder".

**Transaction Amounts** :  
The transaction amounts vary significantly, with the highest being $50 million, showcasing a range of contributions from various individuals and entities.

**Diversity of Contributions** :  
The list includes individuals from different sectors (e.g., investments, aerospace), reflecting a diverse set of contributors.

In [88]:
spark.sql( """
    SELECT CMTE_ID, SUM(TRANSACTION_AMT) AS TOTAL_CONTRIBUTIONS
    FROM contributions
    GROUP BY CMTE_ID
    ORDER BY TOTAL_CONTRIBUTIONS DESC
""").show(10)

+---------+-------------------+
|  CMTE_ID|TOTAL_CONTRIBUTIONS|
+---------+-------------------+
|C00401224|       162589222.00|
|C00744946|        73562149.00|
|C00825851|        71965272.00|
|C00703975|        56540707.00|
|C00694323|        48689346.00|
|C00873893|        39590159.00|
|C00867937|        38814150.00|
|C00571703|        34900100.00|
|C00504530|        32856895.00|
|C00669259|        28961863.00|
+---------+-------------------+
only showing top 10 rows



**Intepretation**

  **Top Committees by Contributions** :  
The committee with the highest total contributions is C00401224, with a substantial amount of $162 million. This indicates that this committee has received significant financial backing, suggesting it plays a major role in its respective political or social activities.

**Significant Financial Support** :  
Other notable committees include C00744946 and C00825851, with total contributions over $7million. These figures reflect strong financial support and suggest that these committees are also influential within their domains.

**Diverse Funding Sources** :  
The list continues with several other committees receiving contributions ranging from approximately 28 to 56 million dollars. This diversity in funding indicates a broad base of support for various initiatives, campaigns, or causes represented by these committees.

**Overall Financial Landscape** :  
The total contributions highlight the competitive nature of fundraising within political or advocacy contexts, showcasing how different committees vie for financial resources to further their objectives.

## Conclusion

In this analysis, we explored the financial contributions made to various political committees, revealing significant patterns and insights regarding funding sources.
Population Influence: States with larger populations tend to have both higher counts and higher total amounts of contributions.

Political Engagement: High contribution counts may indicate robust political engagement and activism within those states.

Financial Impact: The total contribution amounts provide insight into which states are financially supporting political campaigns or causes more heavily.

Average Contribution Size: You can infer that states like New York may have fewer but larger contributions compared to states like California.

The data highlighted that the top committee CMTE_ID C00401224, received a remarkable total of $162 million, indicating it is a major player in the political landscape.     

Similarly, the second committee, C00744946, got approximately $73 million, showcasing substantial financial backing as well.

The contributions reflect a trend where a few committees dominate fundraising efforts, suggesting that these organizations likely have strong networks and strategies for attracting large donations. This concentration of financial resources can significantly influence political campaigns and elections, as committees with greater funding are better positioned to amplify their messages, engage voters, and support candidates.
    
Overall, the findings underscore the critical role of financial contributions in shaping political dynamics and highlight the importance of transparency in campaign financing. Understanding these patterns is essential for stakeholders aiming to navigate the complex interplay between money and politics effectively.

In [89]:
# Stop the SparkContext when done
spark.stop()